In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import chart_studio.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import mysql.connector
from mysql.connector import Error

In [7]:
query_return_table = 'SELECT t.Conv_Date AS "Observation", COUNT(Conv_ID) AS "Total_Orders", COUNT(CASE WHEN t.Customer_Type = "Returning" THEN Conv_ID END) AS "Returning_Orders", COUNT(CASE WHEN t.Customer_Type = "Returning" THEN Conv_ID END)/COUNT(Conv_ID) AS "Repurchase Rate" FROM ( SELECT amo.Hashed_Email, amo.Conv_Date, amo.Conv_ID, CASE WHEN amo.Conv_Date = first_orders.first_date THEN "New" ELSE "Returning" END AS "Customer_Type" FROM api_magento.api_magento_orders amo JOIN ( SELECT Hashed_Email, MIN(Conv_Date) AS "first_date" FROM api_magento.api_magento_orders amo WHERE Account = "Agu" AND Status != "cancelled" GROUP BY 1) AS first_orders ON first_orders.Hashed_Email = amo.Hashed_Email WHERE Account = "Agu" AND Status != "cancelled") AS t GROUP BY 1;'

query_rfm_table = 'SELECT Hashed_Email, COUNT(Conv_ID) AS "frequency", DATEDIFF(CURDATE(),MAX(Conv_Date)) AS "recency", AVG(Revenue_excl_VAT) AS "monetary_value" FROM api_magento.api_magento_orders amo WHERE Account = "Agu" AND Status != "cancelled" GROUP BY 1'

In [8]:
try:
    connection = mysql.connector.connect(host='attribution-system-fsg-new.cob86lv75rzo.eu-west-1.rds.amazonaws.com',
                                         database='api_magento',
                                         user='fsg',
                                         password='Attribution3.0')

    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("Your connected to database: ", record)
        df_return = pd.read_sql(query_return_table,con=connection)
        df_rfm = pd.read_sql(query_rfm_table,con=connection)

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if (connection.is_connected()):
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Connected to MySQL Server version  8.0.16
Your connected to database:  ('api_magento',)
MySQL connection is closed


In [9]:
df_return.shape, df_rfm.shape

((162, 4), (36851, 4))

In [10]:
df_rfm

,Hashed_Email,frequency,recency,monetary_value
0,355fad052994e5aa5fa4c558e1aaf5d5,1,178,40.500
1,d02866520b5fdc371da5e2bb9c605c17,1,178,19.950
2,f83977bcbd360e20200efb7009a9d23d,1,178,37.870
3,28c2ee204baa3740068d6ef834428976,2,175,184.875
4,9b8b0d1ae2d6a45a4597ea486f2781a8,1,178,115.700
...,...,...,...,...
36846,e6f740c1cd4d924a198b2b2320e40640,1,1,66.970
36847,36870f89265fbf06121cc9167faa500f,1,1,40.330
36848,73769f8e5d3b17697d69b54263c363c7,1,1,200.000
36849,e0a0fd773f9fc33a36e775afe5cb4990,1,1,14.830


In [11]:
# Purchase Frequency
purchase_frequency=sum(df_rfm['frequency'])/df_rfm.shape[0]

# Repeat Rate
repeat_rate=df_rfm[df_rfm.frequency > 1].shape[0]/df_rfm.shape[0]

# Churn Rate
churn_rate=1-repeat_rate

# Customer Value
df_rfm['CLV']=(df_rfm['monetary_value']*purchase_frequency)/churn_rate

In [15]:
df_rfm.quantile([.2, .4, .6, .8], axis=0)

,frequency,recency,monetary_value,CLV
0.2,1.0,29.0,25.21,32.896313
0.4,1.0,69.0,40.45,52.782859
0.6,1.0,108.0,57.86,75.501019
0.8,1.0,143.0,100.84,131.585253


In [12]:
purchase_frequency,repeat_rate,churn_rate

(1.1611625193346178, 0.11014626468752545, 0.8898537353124746)

In [13]:
df_rfm.sample(5)

,Hashed_Email,frequency,recency,monetary_value,CLV
28911,350b779068743d57ce4de3c13227b5de,1,35,44.20,57.676202
34173,763b1e4dcd33b6b2cfa3da4b5fa9dc50,3,11,123.97,161.767391
2397,c7d9edb3d23cf2eee78cc54ce22163ea,1,163,109.24,142.546341
581,b076b2abd07b9c3c6a36d0c14d5d64b0,1,176,11.77,15.358572
21725,9180c1069617404550136047c29983ef,2,26,132.04,172.297865


In [14]:
df_rfm.recency.median()

92.0

## OTHER RFM STUFF

In [54]:
rfm['r_score'] = 0
for d in range(len(rfm)):
    if rfm.recency[d] < 60:
        rfm['r_score'][d] = 5
    elif rfm.recency[d] < 190:
        rfm['r_score'][d] = 4
    elif rfm.recency[d] < 400:
        rfm['r_score'][d] = 3
    elif rfm.recency[d] < 610:
        rfm['r_score'][d] = 2
    else:
        rfm['r_score'][d] = 1


In [55]:
rfm['f_score'] = 0
for o in range(len(rfm)):
    if rfm.frequency[o] == 1:
        rfm['f_score'][o] = 1
    elif rfm.frequency[o] == 2:
        rfm['f_score'][o] = 2
    elif rfm.frequency[o] == 3:
        rfm['f_score'][o] = 3
    elif rfm.frequency[o] == 4:
        rfm['f_score'][o] = 4
    else:
        rfm['f_score'][o] = 5
    

In [56]:
rfm['m_score'] = 0
for o in range(len(rfm)):
    if rfm.monetary_value[o] < 33:
        rfm['m_score'][o] = 1
    elif rfm.monetary_value[o] < 43:
        rfm['m_score'][o] = 2
    elif rfm.monetary_value[o] < 50:
        rfm['m_score'][o] = 3
    elif rfm.monetary_value[o] < 61:
        rfm['m_score'][o] = 4
    else:
        rfm['m_score'][o] = 5

In [58]:
rfm['score_conc'] = ''
for s in range(len(rfm)):
    rfm['score_conc'][s] = str(rfm['r_score'][s])+str(rfm['f_score'][s])+str(rfm['m_score'][s])

In [59]:
rfm.head()

,email,recency,frequency,monetary_value,CLV,r_score,f_score,m_score,score_conc
0,003ppcs4s2vcj0q@marketplace.amazon.de,734,3,40.150000,312.997351,1,3,2,132
1,009g57nc9rg76dy@marketplace.amazon.de,32,11,45.452727,354.335821,5,5,3,553
2,00j89nxrhbft07k@marketplace.amazon.de,1060,1,44.810000,349.325313,1,1,3,113
3,00jx5l3sntnj5p1@marketplace.amazon.de,847,1,23.830000,185.771529,1,1,1,111
4,00sk0ls66hd2c5x@marketplace.amazon.de,958,2,27.050000,210.873682,1,2,1,121


In [69]:
rfm['clv_score'] = ''
for d in range(len(rfm)):
    if rfm.CLV[d] < 215:
        rfm['clv_score'][d] = "Low"
    elif rfm.CLV[d] < 277:
        rfm['clv_score'][d] = "Mid-Low"
    elif rfm.CLV[d] < 320:
        rfm['clv_score'][d] = "Mid"
    elif rfm.CLV[d] < 386:
        rfm['clv_score'][d] = "Mid-High"
    elif rfm.CLV[d] >= 386:
         rfm['clv_score'][d] = "High"


In [62]:
promising = rfm[rfm['score_conc'].isin(["525", "524", "523", "522", "521", "515", "514", "513", "425", "424", "413", "414", "415", "315", "314", "313"])].loc[:,['email']]

In [66]:
rfm[rfm['score_conc'].isin(["525", "524", "523", "522", "521", "515", "514", "513", "425", "424", "413", "414", "415", "315", "314", "313"])]

,email,recency,frequency,monetary_value,CLV,r_score,f_score,m_score,score_conc,clv_score
6,00w4f6c1l0dr8wr@marketplace.amazon.de,39,2,36.400,283.763476,5,2,2,522,Mid
7,00y6n4lv9yxzn94@marketplace.amazon.de,343,1,57.710,449.889841,3,1,4,314,High
51,05xzx8jgpk2xdy5@marketplace.amazon.de,115,1,52.480,409.118331,4,1,4,414,High
54,06b2pyv78fyqq3d@marketplace.amazon.de,40,2,49.485,385.770210,5,2,3,523,Mid-High
97,0d79hym02d70vj2@marketplace.amazon.de,196,1,53.470,416.836074,3,1,4,314,High
...,...,...,...,...,...,...,...,...,...,...
23468,zt03hsjr6gx8hqv@marketplace.amazon.de,197,1,58.810,458.465111,3,1,4,314,High
23471,ztbr3mwpwl6zfr2@marketplace.amazon.de,266,1,46.050,358.991980,3,1,3,313,Mid-High
23481,zuechter1@gmx.de,10,1,102.800,801.397950,5,1,5,515,High
23506,zwinger_von_itzenplitz@gmx.de,256,1,108.810,848.250106,3,1,5,315,High


In [63]:
promising.shape

(1531, 1)

In [70]:
clv_relevant = rfm[rfm['clv_score'].isin(["Mid-High", "High"])].loc[:,['email']]
clv_relevant.shape

(8964, 1)

In [68]:
rfm.shape # but in powr bi there are only 15,466 customers

(23530, 10)

In [71]:
promising.to_csv('./Outputs/Promising_Cust_09062021.csv', index=False)

clv_relevant.to_csv('./Outputs/High_MidHigh_CLV_Cust_09062021.csv', index=False)

## Experiments